In [1]:
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 35.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux20

In [ ]:
!pip install --upgrade torch torchvision torchaudio
!pip install torchdata --upgrade
!pip install --upgrade dgl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0+cu121
    Uninstalling torch-2.3.0+cu121:
      Successfully uninstalled torch-2.3.0+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.0+cu121
    Uninstalling torchvision-0.18.0+cu121:
      Successfully uninstalled torchvision-0.18.0+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.0+cu121
    Uninstalling torchaudio-2.3.0+cu121:
      Successfully uninstalled torc

ERROR: Operation cancelled by user


In [1]:
import dgl
import networkx as nx
import matplotlib.pyplot as plt


FileNotFoundError: Cannot find DGL C++ graphbolt library at /usr/local/lib/python3.10/dist-packages/dgl/graphbolt/libgraphbolt_pytorch_2.3.1.so

In [2]:
# Create an empty DGL graph
g = dgl.DGLGraph()

# Add 5 nodes to the graph
g.add_nodes(5)

# Add edges between the nodes (0 -> 1, 1 -> 2, 2 -> 3, 3 -> 4, 4 -> 0)
g.add_edges([0, 1, 2, 3, 4], [1, 2, 3, 4, 0])

print("Number of nodes:", g.number_of_nodes())
print("Number of edges:", g.number_of_edges())


NameError: name 'dgl' is not defined

In [ ]:
# Convert to NetworkX graph for visualization
nx_g = g.to_networkx()

# Draw the graph using NetworkX
pos = nx.spring_layout(nx_g)  # Layout for visualization
nx.draw(nx_g, pos, with_labels=True, node_color=[[0.5, 0.5, 0.5]], node_size=500, edge_color='gray')
plt.show()


In [ ]:
import torch

# Adding node features
g.ndata['feature'] = torch.eye(5)  # Identity matrix as node features

# Adding edge features
g.edata['weight'] = torch.ones(5, 1)  # Edge weights as features

print("Node features:\n", g.ndata['feature'])
print("Edge features:\n", g.edata['weight'])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn

class SimpleGNN(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(SimpleGNN, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, inputs):
        # Initialize the node features
        g.ndata['h'] = inputs

        # Perform message passing
        g.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'h'))

        # Apply a linear transformation
        h = g.ndata['h']
        return self.linear(h)

# Instantiate the model
model = SimpleGNN(5, 2)


In [ ]:
# Node features (identity matrix)
inputs = torch.eye(5)

# Apply the GNN model
outputs = model(g, inputs)
print("GNN outputs:\n", outputs)


In [ ]:
import matplotlib.pyplot as plt

# Convert the outputs to numpy for visualization
embeddings = outputs.detach().numpy()

# Plot the node embeddings
plt.scatter(embeddings[:, 0], embeddings[:, 1])

for i in range(embeddings.shape[0]):
    plt.text(embeddings[i, 0], embeddings[i, 1], str(i), fontsize=12, ha='right')

plt.xlabel('Embedding Dimension 1')
plt.ylabel('Embedding Dimension 2')
plt.title('Node Embeddings')
plt.show()
